In [2]:
%load_ext autoreload
%autoreload 2

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append('/home/hanyuji/Workbench/scSSSD')

import wot
from dataset.dataset_utils import loadSCData

In [4]:
ann_data, cell_tps, cell_types, n_genes, n_tps = loadSCData("zebrafish", "three_interpolation")


ann_data.obs = pd.DataFrame(index=ann_data.obs.index)
ann_data.obs['day'] = cell_tps

[ Data=zebrafish | Split=three_interpolation ] Loading data...


In [5]:
# create OTModel
ot_model = wot.ot.OTModel(ann_data, epsilon=0.05, lambda1=1, lambda2=50)

In [6]:
boundary = 0.09
all_couple_dict = {}

for time_start in range(1, 12):
    tmap_annotated = ot_model.compute_transport_map(time_start, time_start + 1)
    print(f'time:{time_start}, shape:{tmap_annotated.X.shape}')

    matrix_x = tmap_annotated.X
    cell_1 = tmap_annotated.obs.index.tolist()
    cell_2 = tmap_annotated.var.index.tolist()

    single_couple_dict = {}
    for i, item in enumerate(range(matrix_x.shape[0])):
        cell_couple_list = []
        l = matrix_x[item].tolist()
        for j, score in enumerate(l):
            if score > boundary:
                cell_couple_list.append(cell_2[j])
        if len(cell_couple_list) > 0:
            single_couple_dict.update({cell_1[i]: cell_couple_list})

    all_couple_dict.update({time_start: single_couple_dict})

time:1, shape:(311, 200)
time:2, shape:(200, 1158)
time:3, shape:(1158, 1467)
time:4, shape:(1467, 5716)
time:5, shape:(5716, 1026)
time:6, shape:(1026, 4101)
time:7, shape:(4101, 6178)
time:8, shape:(6178, 5442)
time:9, shape:(5442, 7114)
time:10, shape:(7114, 1614)
time:11, shape:(1614, 4404)


In [7]:
trajectories = []

def find_trajectories(start_cell, time_step, trajectory, time_end):
    if time_step == time_end:
        trajectories.append(trajectory)
        return
    if start_cell not in all_couple_dict.get(time_step, {}):
        return
    for next_cell in all_couple_dict[time_step][start_cell]:
        find_trajectories(next_cell, time_step + 1, trajectory + [next_cell], time_end)


time_end = 12
for start_cell in tqdm(all_couple_dict[1]):
    find_trajectories(start_cell, 1, [start_cell], time_end)

# print(trajectories)

100%|██████████| 208/208 [00:00<00:00, 8007.48it/s]


In [8]:
cell_name_traj = np.array(trajectories)
print(f'cell_name_traj: {cell_name_traj.shape}')


# 初始化一个空的列表，用于存储每个时间点的细胞基因表达数据
data_by_timepoint = []

# 遍历每个时间点
for time_point in tqdm(range(cell_name_traj.shape[1])):
    # 获取当前时间点的所有细胞名
    cells_at_timepoint = cell_name_traj[:, time_point]

    # 初始化一个空列表，用于存储当前时间点的所有细胞数据
    cell_data_at_timepoint = []

    # 遍历当前时间点的每个细胞名
    for cell_name in cells_at_timepoint:
        # 检索当前细胞的基因表达数据
        if cell_name in ann_data.obs.index:
            cell_data = ann_data[cell_name, :].X
            cell_data_at_timepoint.append(cell_data.flatten())

    # 将当前时间点的所有细胞数据转换为一个NumPy数组，并添加到data_by_timepoint列表
    if cell_data_at_timepoint:
        data_by_timepoint.append(np.stack(cell_data_at_timepoint))

# 将所有时间点的数据堆叠成一个NumPy数组，其形状应该是 (timepoint, cell_num, gene)
final_array = np.stack(data_by_timepoint)

# 确保最终数组形状为 (timepoint, cell_num, gene)
print("Final array shape:", final_array.shape)

cell_name_traj: (9582, 12)


100%|██████████| 12/12 [00:36<00:00,  3.04s/it]


Final array shape: (12, 9582, 2000)


In [9]:
# 归一化 除以5
# final_array = final_array / 5

In [10]:
np.save('/mnt/sdb/hanyuji-data/SSSD_results/wot_result/gene_traj_009_by5.npy', final_array)

In [1]:
final_array.shape

NameError: name 'final_array' is not defined